# Case Study 1

## Business Understanding
You should always state the objective at the beginning of every case (a guideline you should follow in real life as well) and provide some initial "Business Understanding" statements (i.e., what is trying to be solved for and why might it be important)

build a linear regression model using L1 or L2 regularization (or both) to predict the critical temperature. In addition, include in your write-up which variable carries the most importance.

## Data Evaluation and Engineering
Summarize the data being used in the case using appropriate mediums (charts, graphs, tables); address questions such as: Are there missing values? Which variables are needed (which ones are not)? What assumptions or conclusions are you drawing that need to be relayed to your audience?

In [1]:
import numpy as np
import pandas as pd

# Load Data
data_train = pd.read_csv('./data/train.csv')
data_materials = pd.read_csv('./data/unique_m.csv')

# Drop the duplicate column 'critical_temp' in the

data_train = data_train.drop(['critical_temp'], axis=1)

# Merge the two frames
data = pd.merge(data_train, data_materials, left_index=True, right_index=True)

data.info()
data.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21263 entries, 0 to 21262
Columns: 169 entries, number_of_elements to material
dtypes: float64(156), int64(12), object(1)
memory usage: 27.4+ MB


,number_of_elements,mean_atomic_mass,wtd_mean_atomic_mass,gmean_atomic_mass,wtd_gmean_atomic_mass,entropy_atomic_mass,wtd_entropy_atomic_mass,range_atomic_mass,wtd_range_atomic_mass,std_atomic_mass,...,Pt,Au,Hg,Tl,Pb,Bi,Po,At,Rn,critical_temp
count,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,...,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.000000,21263.0,21263.0,21263.0,21263.000000
mean,4.115224,87.557631,72.988310,71.290627,58.539916,1.165608,1.063884,115.601251,33.225218,44.391893,...,0.034108,0.020535,0.036663,0.047954,0.042461,0.201009,0.0,0.0,0.0,34.421219
std,1.439295,29.676497,33.490406,31.030272,36.651067,0.364930,0.401423,54.626887,26.967752,20.035430,...,0.307888,0.717975,0.205846,0.272298,0.274365,0.655927,0.0,0.0,0.0,34.254362
min,1.000000,6.941000,6.423452,5.320573,1.960849,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000210
25%,3.000000,72.458076,52.143839,58.041225,35.248990,0.966676,0.775363,78.512902,16.824174,32.890369,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,5.365000
50%,4.000000,84.922750,60.696571,66.361592,39.918385,1.199541,1.146783,122.906070,26.636008,45.123500,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,20.000000
75%,5.000000,100.404410,86.103540,78.116681,73.113234,1.444537,1.359418,154.119320,38.356908,59.322812,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,63.000000
max,9.000000,208.980400,208.980400,208.980400,208.980400,1.983797,1.958203,207.972460,205.589910,101.019700,...,5.800000,64.000000,8.000000,7.000000,19.000000,14.000000,0.0,0.0,0.0,185.000000


In [2]:
# Columns with missing data?
print(data.columns[data.isnull().any()])

Index([], dtype='object')


In [3]:
# Columns with a Constant value
data.columns[data.nunique() <= 1]

Index(['He', 'Ne', 'Ar', 'Kr', 'Xe', 'Pm', 'Po', 'At', 'Rn'], dtype='object')

In [4]:
# Drop columns with constant values
data.drop(columns=['material', 'He', 'Ne', 'Ar', 'Kr', 'Xe', 'Pm', 'Po', 'At', 'Rn'], inplace=True)
print(data.shape)

(21263, 159)


In [5]:
# Create our standard numpy stuff
X = data.drop(columns=['critical_temp']).values
y = data.loc[:,'critical_temp'].values

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.pipeline import make_pipeline

X_train, X_test, y_train, y_test =\
    train_test_split(X, y,
    test_size=0.2,
    random_state=1)

pipe_svc = make_pipeline(StandardScaler(), Lasso(random_state=1))

param_range = [0.001, 0.01, 0.1, 1, 10, 100, 1000]

param_grid = [{'lasso__alpha': param_range}]

gs = GridSearchCV(estimator=pipe_svc, param_grid=param_grid, scoring='r2', cv=5, n_jobs=-1)

gs = gs.fit(X_train, y_train)

print(gs.best_score_)
print(gs.best_params_)

0.5552290552110917
{'lasso__alpha': 1}


In [7]:
#print()
#print(gs.best_estimator_['lasso'].coef_)
idx = 0

for x in gs.best_estimator_['lasso'].coef_:
    print(data.columns[idx], x)
    idx += 1

number_of_elements 0.09558044669552052
mean_atomic_mass -0.0
wtd_mean_atomic_mass -0.0
gmean_atomic_mass -0.0
wtd_gmean_atomic_mass -0.0
entropy_atomic_mass 0.0
wtd_entropy_atomic_mass 4.064872033009709
range_atomic_mass 1.2665770184465959
wtd_range_atomic_mass -0.0
std_atomic_mass 0.0
wtd_std_atomic_mass -0.0
mean_fie 0.07391944793987394
wtd_mean_fie 0.0
gmean_fie 0.0
wtd_gmean_fie 0.0
entropy_fie 0.0
wtd_entropy_fie 0.0
range_fie 1.8755749458833757
wtd_range_fie -0.0
std_fie 0.0
wtd_std_fie 0.0
mean_atomic_radius -0.0
wtd_mean_atomic_radius 0.0
gmean_atomic_radius -0.0
wtd_gmean_atomic_radius -0.0
entropy_atomic_radius 0.0
wtd_entropy_atomic_radius 0.0
range_atomic_radius 1.172870788924883
wtd_range_atomic_radius -0.0
std_atomic_radius 0.0
wtd_std_atomic_radius 0.0
mean_Density -0.0
wtd_mean_Density -0.0
gmean_Density -1.9322980234576133
wtd_gmean_Density -0.0
entropy_Density -0.0
wtd_entropy_Density 0.0
range_Density 0.0
wtd_range_Density -0.0
std_Density -0.0
wtd_std_Density -0.0
m

In [8]:
## install pandas 1.2.4
## pip install pandas-profiling==2.8.0

from pandas_profiling import ProfileReport

profile = ProfileReport(data, title="Pandas Profiling Report", minimal=True)

profile.to_file(output_file="PandasProfile.html")

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 62.67it/s]


## Modeling Preparations
Which methods are you proposing to utilize to solve the problem?  Why is this method appropriate given the business objective? How will you determine if your approach is useful (or how will you differentiate which approach is more useful than another)?  More specifically, what evaluation metrics are most useful given that the problem is a regression one (ex., RMSE, logloss, MAE, etc.)?

## Model Building and Evaluation
In this case, your primary task is to build a linear regression model using L1 or L2 regularization (or both) to predict the critical temperature and will involve the following steps:

- Specify your sampling methodology
- Setup your model(s) - specifying the regularization type chosen and including the parameters utilized by the model
- Analyze your model's performance - referencing your chosen evaluation metric (including supplemental visuals and analysis where appropriate)

## Model Interpretability & Explainability
Using at least one of your models above (if multiple were trained):

- Which variable(s) was (were) ""most important"" and why?  How did you come to the conclusion and how should your audience interpret this?

## Case Conclusions
After all of your technical analysis and modeling; what are you proposing to your audience and why?  How should they view your results and what should they consider when moving forward?  Are there other approaches you'd recommend exploring?  This is where you "bring it all home" in language they understand.